# Combinatorial Species and Mathematical Biology

## Trees

Recall the basic species.

* $1$ is the empty set species

* $X$ is the singleton species

* $E$ is the set species

### Rooted trees

Let's try to define the species $\mathcal A$ of rooted trees on $n$ nodes. Each tree consists of a single **root**, and a **set** of root subtrees. In the "functional equation" language of species this becomes

$$
\mathcal A = X \cdot E(\mathcal A).
$$


In [ ]:
from sage.combinat.species.library import *

One = EmptySetSpecies()
X = SingletonSpecies()
E = SetSpecies()

# define A
A = 

This allows us to perform labeled and unlabeled enumeration.

In [ ]:
As = A.generating_series()

print('generating series:')
print(As)

Al = [factorial(i)*As.coefficient(i) for i in range(1,11)]

print('counts of labeled structures:')
print(Ac)

Ais = A.isotype_generating_series()

print('isotype generating series:')
print(Ais)

Au = Ais[:11]

print('counts of unlabeled structures:')
print(Au)

In [ ]:
# search OEIS for Al

In [ ]:
# search OEIS for Au

Searching OEIS gives us the right sequence IDs as the first option. In particular, the number of rooted labeled trees is $n^{n-1}$.

### Binary rooted trees

Now let's try planar binary trees $\mathcal B$. There is one tree on the empty set, and otherwise a tree is a **root** node, together with a **left subtree** and the **right subtree**

In [ ]:
# define B
B = 

In [ ]:
Bis = B.isotype_generating_series()
Bu = Bis[:11]

print('isotype generating series:')
print(Bis)

In [ ]:
# search OEIS

## Phylogenetic trees

But what about phylogenetic trees? The leaves are labeled but the internal nodes aren't.

We can solve this:

$$ \mathcal T = X + E_2(\mathcal T),$$

where $E_2$ is a species of pairs. Intuitively, a tree here is either a leaf or an unlabeled root together with a pair of subtrees.

In [ ]:
E = SetSpecies()
E2 = E.restricted(min=2, max=3)

T = 

In [ ]:
Tis = T.isotype_generating_series()
Tu = Tis[:17]

print('isotype generating series:')
print(Tis)

In [ ]:
# search OEIS for Tu

Any trees can be defined like this. For example, here is the multifurcating tree species:

In [ ]:
E2p = E.restricted(min=2)

Tmult = CombinatorialSpecies(min=1)
Tmult.define(X + E2p(Tmult))

In [ ]:
# lookup in OEIS again
oeis(Tmult.isotype_generating_series()[:10])

## Tanglegrams

In a certain way, tanglegrams are simply pairs of trees, but it's hard to come up with a correct representation of tanglegrams in the language of combinatorial species.

Gessel (2016) introduces a new combinator for species --- the **cartesian product** --- to encode tanglegrams.

Recall that the cycle index series for a species $F$ is defined as 

$$
Z_F = \sum_{n=0}^\infty \frac{1}{n!} \sum_{\sigma \in \mathcal S_n} \mathrm{fix}\,F[\sigma]\,p_\sigma
$$

with $p_\sigma = x_1^{\sigma_1}x_2^{\sigma_2}\cdots$. Noting that $\mathrm{fix}\,F[\sigma]$ depends only on the cycle type (conjugacy class) of $\sigma$, we can rewrite $Z_F$ as 

$$
Z_F  \sum_{n=0}^\infty \sum_{\lambda \vdash n} \frac{\mathrm{fix}\,F[\lambda]}{z_\lambda} p_\lambda,
$$
where $\lambda$ is a partition of $n$, $z_\lambda = 1^{m_1}m_1!\,2^{m_2} m_2!\cdots$, and $p_\lambda = x_1^{m_1}x_2^{m_2}\cdots$. This is exactly the representation that is used by `Sage`.

With this, the cartesian product of two species $F\times G$ is defined as a *pair of objects on the same set*, and 

$$
Z_{F\times G} = \sum_{n=0}^\infty \sum_{\lambda \vdash n} \frac{\mathrm{fix}\,F[\lambda] \ast \mathrm{fix}\,G[\lambda]}{z_\lambda} p_\lambda
$$

With this, the species of tanglegrams of binary trees is defined as 

$$
\mathrm{Tang} = \mathcal T\times \mathcal T.
$$

Cartesian product is not implemented in `Sage`, but we can do it ourselves.

In [ ]:
def cartprod_isotypes(F, G, n=17):
    # extract cycle index series Z_F, G_F
    ZF = F.cycle_index_series()
    ZG = G.cycle_index_series()
    
    result = []
    
    # for terms of each degree,
    # extract the coefficients r_lambda and multiply them together
    for deg in range(1,n):
        
        val = 0
        
        # each partition corresponds to a monomial in the cycle index series
        for p_lambda in Partitions(deg).list():
            z_lambda = Partition(p_lambda).aut()
            
            r_lambda_F = ZF.coefficient(deg).coefficient(p_lambda) * z_lambda
            r_lambda_G = ZG.coefficient(deg).coefficient(p_lambda) * z_lambda
            
            val += (r_lambda_F * r_lambda_G) / z_lambda

        result.append(val)
        
    return(result)

In [ ]:
cartprod_isotypes(T,T)

In [ ]:
oeis(cartprod_isotypes(T,T))

In [ ]:
cartprod_isotypes(Tmult,Tmult)

In [ ]:
oeis(cartprod_isotypes(Tmult,Tmult))

## Bonus: a different problem

Suppose you look at genotypes of $n$ diploid individuals at some locus. Some of the observed $2n$ alleles match. How many identity configurations are there?


<img style="width: 600px;" src="figure1.jpg" />

Looking at this closely, among the points there is a **set** of nonmatcing alleles, and possibly **bunch of sets** of matching alleles.

This immediately suggests the following definition:

$$ 
\mathcal I = E + E \cdot E_{1+} (E_{2+})
$$
where $E_{1+}$ is the species of sets of cardinality $\geq 1$, and $E_{2+}$ is the species of sets of cardinality $\geq 2$. 

This definition says that $\mathcal I$ is either a set of alleles none of which match another ($E$), **or** a partition of alleles into set of nonmatching alleles ($E$) and at least one set ($E_{1+}$) of matching alleles ($E_{2+}$).

Let's compute!

In [ ]:
E1p = E.restricted(min=1)

sp = CombinatorialSpecies()
sp.define(E + E*E1p(E2p))

In [ ]:
fi = sp.isotype_generating_series()
fi

In [ ]:
oeis(fi[:10])

In [ ]:
f = sp.generating_series()
f

In [ ]:
fcoef = [x*factorial(i+1) for i, x in enumerate(sp.generating_series()[1:16])]
fcoef

In [ ]:
oeis(fcoef)

Aha! We have discovered... partitions!